<a href="https://colab.research.google.com/github/Phatdeluxe/DS-Unit-2-Regression-Classification/blob/master/module2/assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module2')

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [63]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,description_given,cats_and_dogs,description_characters
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,588.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,8.0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,691.0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,492.0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,479.0


In [20]:
df.isnull().sum()

bathrooms                  0
bedrooms                   0
created                    0
description             1425
display_address          133
latitude                   0
longitude                  0
price                      0
street_address            10
interest_level             0
elevator                   0
cats_allowed               0
hardwood_floors            0
dogs_allowed               0
doorman                    0
dishwasher                 0
no_fee                     0
laundry_in_building        0
fitness_center             0
pre-war                    0
laundry_in_unit            0
roof_deck                  0
outdoor_space              0
dining_room                0
high_speed_internet        0
balcony                    0
swimming_pool              0
new_construction           0
terrace                    0
exclusive                  0
loft                       0
garden_patio               0
wheelchair_access          0
common_outdoor_space       0
dtype: int64

In [10]:
df['created'] = pd.to_datetime(df['created'])
df['created'].head()

0   2016-06-24 07:54:24
1   2016-06-12 12:19:27
2   2016-04-17 03:26:41
3   2016-04-18 02:22:02
4   2016-04-28 01:32:41
Name: created, dtype: datetime64[ns]

In [93]:
(df['created'].dt.month < 4).sum()

0

In [0]:
import numpy as np
df['description_given'] = (df['description'].isnull() == False).astype(int)
df['cats_and_dogs'] = (df['cats_allowed'] & df['dogs_allowed'] == 1).astype(int)
df['description_characters'] = df['description'].str.len()
df['description_characters'] = df['description_characters'].replace({np.NaN: 0, np.inf: 0, -np.inf: 0})

In [74]:
df['description_characters'].describe()

count    48817.000000
mean       601.806215
std        391.826042
min          0.000000
25%        341.000000
50%        564.000000
75%        809.000000
max       4466.000000
Name: description_characters, dtype: float64

In [0]:
train = df[df['created'].dt.month <= 5 ]
test = df[df['created'].dt.month == 6]

y_train = train['price']
y_test = test['price']

In [0]:
import itertools
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

def regression_3d(df, x, y, z, num=100, **kwargs):
  fig = px.scatter_3d(df, x, y, z, **kwargs)

  features = [x, y]
  target = z
  model = LinearRegression()
  model.fit(df[features], df[target])

  xmin, xmax = df[x].min(), df[x].max()
  ymin, ymax = df[y].min(), df[y].max()
  xcoords = np.linspace(xmin, xmax, num)
  ycoords = np.linspace(ymin, ymax, num)
  coords = list(itertools.product(xcoords, ycoords))

  predictions = model.predict(coords)
  Z = predictions.reshape(num, num).T

  fig.add_trace(go.Surface(x=xcoords, y=ycoords, z=Z))

  return fig

In [90]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model1 = LinearRegression()

features = ['bedrooms', 'bathrooms']
target = 'price'
print(f'Linear Regression dependent on {features}.\n')

X_train = train[features]
X_test = test[features]

model1.fit(X_train, y_train)
y_pred_train = model1.predict(X_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
print(f'Train Error: {mae_train:.2f} percentage points.\n')
print(f'Train Mean Squared Error: {mean_squared_error(y_train, y_pred_train)}.\n')
print(f'Train R^2: {r2_score(y_train, y_pred_train)}.\n')

y_pred_test = model1.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f'Test Error: {mae_test:.2f} percentage points.\n')
print(f'Test Mean Squared Error: {mean_squared_error(y_test, y_pred_test)}.\n')
print(f'Test R^2: {r2_score(y_test, y_pred_test)}.\n')
print(f'Model Intercept: {model.intercept_}.\n')
coefficiants = pd.Series(model.coef_, features)
print(f'Coefficents:\n{coefficiants.to_string()}')


Linear Regression dependent on ['bedrooms', 'bathrooms'].

Train Error: 818.53 percentage points.

Train Mean Squared Error: 1517879.6665142525.

Train R^2: 0.5111543084316605.

Test Error: 825.90 percentage points.

Test Mean Squared Error: 1487715.3104108903.

Test R^2: 0.5213303957090346.

Model Intercept: 485.71869002322273.

Coefficents:
bedrooms      389.324896
bathrooms    2072.610116


In [81]:
regression_3d(
    train,
    x='bathrooms',
    y='bedrooms',
    z='price'
)

In [67]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,description_given,cats_and_dogs,description_characters
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,588.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,8.0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,691.0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,492.0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,479.0


In [95]:
features = ['description_given', 'cats_and_dogs']

X_train = train[features]
X_test = test[features]
print(f'Linear Regression dependent on {features}.\n')

X_train = train[features]
X_test = test[features]

model1.fit(X_train, y_train)
y_pred_train = model1.predict(X_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
print(f'Train Error: {mae_train:.2f} percentage points.\n')
print(f'Train Mean Squared Error: {mean_squared_error(y_train, y_pred_train)}.\n')
print(f'Train R^2: {r2_score(y_train, y_pred_train)}.\n')

y_pred_test = model1.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f'Test Error: {mae_test:.2f} percentage points.\n')
print(f'Test Mean Squared Error: {mean_squared_error(y_test, y_pred_test)}.\n')
print(f'Test R^2: {r2_score(y_test, y_pred_test)}.\n')
print(f'Model Intercept: {model.intercept_}.\n')
coefficiants = pd.Series(model.coef_, features)
print(f'Coefficents:\n{coefficiants.to_string()}')

Linear Regression dependent on ['description_given', 'cats_and_dogs'].

Train Error: 1197.19 percentage points.

Train Mean Squared Error: 3092972.1891615624.

Train R^2: 0.0038827436930241044.

Test Error: 1193.79 percentage points.

Test Mean Squared Error: 3097410.820037734.

Test R^2: 0.00341389163728556.

Model Intercept: 485.71869002322273.

Coefficents:
description_given     389.324896
cats_and_dogs        2072.610116


In [0]:
regression_3d(
    train,
    x='cats_and_dogs',
    y='description_given',
    z='price'
)

In [94]:
features = ['description_characters', 'cats_and_dogs']

X_train = train[features]
X_test = test[features]
print(f'Linear Regression dependent on {features}.\n')

X_train = train[features]
X_test = test[features]

model1.fit(X_train, y_train)
y_pred_train = model1.predict(X_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
print(f'Train Error: {mae_train:.2f} percentage points.\n')
print(f'Train Mean Squared Error: {mean_squared_error(y_train, y_pred_train)}.\n')
print(f'Train R^2: {r2_score(y_train, y_pred_train)}.\n')

y_pred_test = model1.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f'Test Error: {mae_test:.2f} percentage points.\n')
print(f'Test Mean Squared Error: {mean_squared_error(y_test, y_pred_test)}.\n')
print(f'Test R^2: {r2_score(y_test, y_pred_test)}.\n')
print(f'Model Intercept: {model.intercept_}.\n')
coefficiants = pd.Series(model.coef_, features)
print(f'Coefficents:\n{coefficiants.to_string()}')

Linear Regression dependent on ['description_characters', 'cats_and_dogs'].

Train Error: 1186.58 percentage points.

Train Mean Squared Error: 3019133.5803881222.

Train R^2: 0.027663077909628186.

Test Error: 1175.75 percentage points.

Test Mean Squared Error: 3010509.836273403.

Test R^2: 0.0313741197937597.

Model Intercept: 485.71869002322273.

Coefficents:
description_characters     389.324896
cats_and_dogs             2072.610116


In [79]:
regression_3d(
    train,
    x='cats_and_dogs',
    y='description_characters',
    z='price'
)